In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [72]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


### Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

* для групп normal (control) и early neoplasia (treatment)
* для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [9]:
from scipy import stats

In [22]:
def ttest_by_gens(control, treatment):
    gen_scores = {}
    for gen in control.columns[2:]:
        gen_scores[gen] = stats.ttest_ind(control[gen], treatment[gen], equal_var=False)[1]
    return gen_scores

In [26]:
%%time
norm_earn_test = ttest_by_gens(data[data.Diagnosis == 'normal'], data[data.Diagnosis == 'early neoplasia'])

Wall time: 5.97 s


In [42]:
%%time
earn_cancer_test = ttest_by_gens(data[data.Diagnosis == 'early neoplasia'], data[data.Diagnosis == 'cancer'])

Wall time: 5.98 s


In [31]:
def valuable_count(gen_test, alpha):
    v_count = 0
    for val in gen_test.values():
        if val < alpha:
            v_count += 1
    return v_count

Часть 1.1: Количество стат. значимых отличий при сравнении normal и early neoplasia без поправки на множественное сравнение

In [46]:
valuable_count(norm_earn_test, 0.05)

1575

Часть 1.2: Количество стат. значимых отличий при сравнении early neoplasia и cancer без поправки на множественное сравнение

In [47]:
valuable_count(earn_cancer_test, 0.05)

3490

### Часть 2: поправка методом Холма

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

### Обратите внимание, что

* применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
* при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [48]:
import statsmodels.stats.multitest as smm

In [63]:
reject1, p_corrected1, alphSidac1, alphBonf1 = smm.multipletests(list(norm_earn_test.values()), 0.025, method='holm')
reject2, p_corrected2, alphSidac2, alphBonf2 = smm.multipletests(list(earn_cancer_test.values()), 0.025, method='holm')

In [67]:
unique1, counts1 = np.unique(reject1, return_counts=True)
dict(zip(unique1, counts1))

{False: 15746, True: 2}

In [68]:
unique2, counts2 = np.unique(reject2, return_counts=True)
dict(zip(unique2, counts2))

{False: 15669, True: 79}

In [101]:
def fold_change(C, T):
    if T > C:
        return T / C
    else:
        return -C / T

In [123]:
def valuable_fc_count(control, treatment, gen_indeces, threshold):
    fc_count = 0
    for idx in gen_indeces:
        if np.abs(fold_change(control.iloc[:, idx].mean(), treatment.iloc[:, idx].mean())) > threshold:
            fc_count += 1
    return fc_count

Часть 2.1: Количество значимых отличий c abs(fold change) > 1.5 при сравнении normal и early neoplasia при поправке Холма-Бонферрони

In [125]:
valuable_fc_count(data[data.Diagnosis == 'normal'], 
                  data[data.Diagnosis == 'early neoplasia'], 
                  np.where(reject1 == True)[0] + 2, 
                  1.5)

2

Часть 2.2: Количество значимых отличий c abs(fold change) > 1.5 при сравнении early neoplasia и cancer при поправке Холма-Бонферрони

In [124]:
valuable_fc_count(data[data.Diagnosis == 'early neoplasia'], 
                  data[data.Diagnosis == 'cancer'], 
                  np.where(reject2 == True)[0] + 2, 
                  1.5)

77

### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [127]:
reject1, p_corrected1, alphSidac1, alphBonf1 = smm.multipletests(list(norm_earn_test.values()), 0.025, method='fdr_bh')
reject2, p_corrected2, alphSidac2, alphBonf2 = smm.multipletests(list(earn_cancer_test.values()), 0.025, method='fdr_bh')

In [128]:
unique1, counts1 = np.unique(reject1, return_counts=True)
dict(zip(unique1, counts1))

{False: 15744, True: 4}

In [129]:
unique2, counts2 = np.unique(reject2, return_counts=True)
dict(zip(unique2, counts2))

{False: 14916, True: 832}

Часть 3.1: Количество значимых отличий c abs(fold change) > 1.5 при сравнении normal и early neoplasia при поправке Бенджамини-Хохберга

In [130]:
valuable_fc_count(data[data.Diagnosis == 'normal'], 
                  data[data.Diagnosis == 'early neoplasia'], 
                  np.where(reject1 == True)[0] + 2, 
                  1.5)

4

Часть 3.2: Количество значимых отличий c abs(fold change) > 1.5 при сравнении early neoplasia и cancer при поправке Бенджамини-Хохберга

In [131]:
valuable_fc_count(data[data.Diagnosis == 'early neoplasia'], 
                  data[data.Diagnosis == 'cancer'], 
                  np.where(reject2 == True)[0] + 2, 
                  1.5)

524